In [ ]:
'''
Mount Google Drive, copy data to runtime, and unzip folders

Make sure to put a link to "EC 523 Project" in your main google drive!
'''

from google.colab import drive
drive.mount('/content/drive')

! cp /content/drive/MyDrive/'Deep Learning Proj'/train.zip /content
! cp /content/drive/MyDrive/'Deep Learning Proj'/test.zip /content
! cp /content/drive/MyDrive/'Deep Learning Proj'/val.zip /content
! cp /content/drive/'My Drive'/'Deep Learning Proj'/math.txt /content

# from path will differ depending on where you saved the zip file in Google Drive
! unzip -DD -q  /content/train.zip -d  /content/
! unzip -DD -q  /content/test.zip -d  /content/
! unzip -DD -q  /content/val.zip -d  /content/



Mounted at /content/drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
'''
Length of datasets
'''
num_train_str = !ls train | wc -l
num_test_str = !ls test | wc -l
num_val_str = !ls val | wc -l
num_train = int(num_train_str[0])
num_test = int(num_test_str[0])
num_val = int(num_val_str[0])

print(f'Number of train images: {num_train}\nNumber of test images: {num_test}\nNumber of validation images: {num_val}\nTotal images: {num_train+num_test+num_val}')

Number of train images: 158480
Number of test images: 30637
Number of validation images: 6765
Total images: 195882


In [ ]:
train_root = "/content/train/"
test_root = "/content/test/"
val_root = "/content/val/"
label_file = "/content/math.txt"

Creating dataset class for images and labels

In [ ]:
import os
import cv2
import torch.utils.data
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np


class LatexDataset(torch.utils.data.Dataset):
  def __init__(self, transform=None, dataroot=val_root): # can change dataroot to be either train_root, test_root, val_root
        '''Initialize the dataset.'''
        self.transform = transform
        self.dataroot = dataroot
        self.labels_txt = label_file
        self._parse()

  def _parse(self):
        '''
        Parse the math.txt file.
        Populates the following private variables:
        - self.im_paths: A list of strings storing the associated image paths
        - self.labels: A list of strings, where each string is the latex code for an image
        '''
        def getImPath(idx):
            # Find image in either train, test, or validation folder
            imname = str(idx - 1).zfill(7) + '.png'
            if os.path.exists(f'{self.dataroot}{imname}'):
              impath = f'{self.dataroot}{imname}'
            else:
              return None

            try:
                Image.open(impath).verify()
            except Exception as e:
                # Some images can't be opened
                # print(f"Image at path {impath} is corrupted. Error: {e}")
                return None

            return impath

        self.im_paths = []
        self.labels = []

        with open(self.labels_txt) as f:
            for idx, line in enumerate(f):
                impath = getImPath(idx+1)

                if impath is not None:
                    labels = line.strip('\n')
                    if len(labels) < 700:
                      self.im_paths.append(impath)            # Image name
                      self.labels.append(labels)    # String of latex code


  def __len__(self):
        '''Return length of the dataset.'''
        assert len(self.labels) == len(self.im_paths)
        return len(self.labels)

  def __getitem__(self, index):
        '''
        Return the (image, attributes) tuple.
        This function gets called when you index the dataset.
        '''
        def img_load(index):
            # print(self.im_paths[0:10])
            imraw = Image.open(self.im_paths[index])
            imgray = imraw.convert('L')                         # Convert image to greyscale
            # print("imgray=")
            # print(list(imgray.getdata()))
            imthresh = imgray.point(lambda p: p > 240 and 255)  # Threshold image to remove background (white)
            # img_array = np.array(imthresh)
            # plt.imshow(img_array)
            # plt.axis('off')  # Turn off axis labels
            # plt.show()
            # print("imthresh=")
            # print(imthresh.getdata())
            if self.transform is not None:
              im = self.transform(imthresh)
            else:
              im = imthresh
            return im

        target = self.labels[index]
        return img_load(index), target

Creating dictionary for vocab and tokens

In [ ]:
'''
Dictionary block: converts a LaTeX string to a dictionary of latex tokens, where
each unique token has its own entry and integer value assigned to it

'''
class LatexDict():
    def __init__(self, num_tokens=256):
        self.labels_txt = label_file
        self.num_tokens = num_tokens
        self.latex_dict = {'<UKN>':0, '<PAD>':1} # Initialize with token for unknown and for padding
        self.latex_dict_inverse = {0:'<UKN>', 1:'<PAD>'} # Initialize inverse dict for quicker reverse lookups
        self.create_dict()

    def create_dict(self):
        # Go through entire label file and populate dictionary
        with open(self.labels_txt) as f:
            for line in f:
                tokens = line.split()
                for token in tokens:
                    if token not in self.latex_dict:
                        # Assign a new ID for the unseen token
                        new_id = len(self.latex_dict)
                        self.latex_dict[token] = new_id
                        self.latex_dict_inverse[new_id] = token

    def map_tokens(self, tex_str_list, batch_size):
        ids_tensor = torch.full((batch_size, self.num_tokens), self.latex_dict['<PAD>'], dtype=torch.float32)

        for row, tex_str in enumerate(tex_str_list):
            tokens = tex_str.split()
            for col, token in enumerate(tokens):
                ids_tensor[row, col] = self.latex_dict[token]

        return ids_tensor

    def tokens_to_tex(self, token_vec):
        tex_str = ' '
        for token_id in token_vec.tolist():
            if token_id in self.latex_dict_inverse:
                if self.latex_dict_inverse[token_id] != '<PAD>' and self.latex_dict_inverse[token_id] != '<UKN>':
                    tex_str += self.latex_dict_inverse[token_id] + ' '

        return tex_str

    def __dict__(self):
        return self.latex_dict

    def __len__(self):
        return len(self.latex_dict)

latex_dict = LatexDict()

In [ ]:
dic = latex_dict.__dict__()
print(dic["<PAD>"])

1


In [ ]:
print(dic.__len__())

697


In [ ]:
dictkeys = list(dic.keys())
print(len(dictkeys))
for i in range(10):
    print(dictkeys[i], end = " : ")
    print(dic[dictkeys[i]])


697
<UKN> : 0
<PAD> : 1
\int : 2
_ : 3
{ : 4
- : 5
\epsilon : 6
} : 7
^ : 8
\infty : 9


In [ ]:
dataset = LatexDataset()

In [ ]:
print(dataset[0])
print(dataset[1])
print(dataset[2])
print(dataset[3])
print(dataset[4])
print(dataset[5])

(<PIL.Image.Image image mode=L size=448x32 at 0x7BA871371060>, 'S \\sim \\tilde { \\psi } Q _ { o } \\tilde { \\psi } + g _ { s } ^ { 1 / 2 } \\tilde { \\psi } ^ { 3 } + \\tilde { \\phi } Q _ { c } \\tilde { \\phi } + g _ { s } \\tilde { \\phi } ^ { 3 } + \\tilde { \\phi } B ( g _ { s } ^ { 1 / 2 } \\tilde { \\psi } ) + \\cdots .')
(<PIL.Image.Image image mode=L size=480x32 at 0x7BA871372140>, 'e ^ { A } = e ^ { A _ { 0 } } \\left( t _ { 0 } - \\mathrm { s i g n } ( m ) t \\right) ^ { - \\frac { m } { 2 } } \\; , \\; \\; \\; \\; \\chi = \\chi _ { 0 } \\left( t _ { 0 } - \\mathrm { s i g n } ( m ) t \\right) ^ { m } \\; ,')
(<PIL.Image.Image image mode=L size=576x64 at 0x7BA8590647C0>, 'g _ { J _ { 1 } \\, J _ { 2 } } ^ { J } \\bigl ( J _ { 1 } , M _ { 1 } ; J _ { 2 } , M _ { 2 } \\vert J _ { 1 } , J _ { 2 } ; J , M _ { 1 } + M _ { 2 } \\bigr ) \\, \\Bigl ( \\xi _ { M _ { 1 } + M _ { 2 } } ^ { ( J ) } ( \\sigma _ { 1 } ) + \\mathrm { d e s c e n d a n t s } \\Bigr ) \\Bigr \\} ,')
(<PIL

CNN Block with 512 channel output

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torch.optim as optim
import math

class CNN_Block(nn.Module):
    def __init__(self):
        super(CNN_Block, self).__init__()
        # self.conv1 = nn.Conv2d(3, 64, 3)
        self.conv1 = nn.Conv2d(1, 64, 3, padding=1)    # Images are originally one channel, added padding as well
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(64, 128, 3, padding=1)
        # self.pool2 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(128, 256, 3, padding=1)
        # self.pool3 = nn.MaxPool2d(2, 2)
        self.conv4 = nn.Conv2d(256, 256, 3, padding=1)
        self.conv5 = nn.Conv2d(256, 512, 3, padding=1)
        # self.pool4 = nn.MaxPool2d(2, 2)

        #self.fc1 = nn.Linear(256 * 2 * 8, 1024)  # Adjusted input size based on the output size of the convolutional layers
        #self.fc2 = nn.Linear(1024, 512)


    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.pool(F.relu(self.conv4(x)))
        x = F.relu(self.conv5(x))
        #print('x: shape:', x.shape)
        #x = x.view(64,-1)   # Flatten so this can be used in linear layers

        #x = F.relu(self.fc1(x))
        #x = self.fc2(x)

        return x


Defining decoding class: \\
  hidden_size: dimensionality of the hidden layer of the LSTM \\
  batch_size: Size of the batch \\
  seq_length: length of the input sequence \\
  vocab_size: size of our dictionary \\
  emd_size: Number of rows in embedding vector to represent \\
  enc_out: Number of output channels in encoder \\
  

In [ ]:
class decoder(nn.Module):
  def __init__(self, hidden_size, batch_size, seq_length, vocab_size, embedding_size, enc_out = 512):
    super(decoder, self).__init__()

    self.hidden_size = hidden_size
    self.batch_size = batch_size
    self.seq_length = seq_length
    self.vocab_size = vocab_size
    self.embedding_size = embedding_size
    self.enc_out = enc_out


    self.cnn_encoder = CNN_Block()
    self.lstm = nn.LSTMCell(hidden_size + embedding_size, hidden_size)
    self.emb = nn.Embedding(vocab_size,embedding_size )

    self.wh_in = nn.Linear(enc_out, hidden_size)
    self.wc_in = nn.Linear(enc_out, hidden_size)
    self.wo_in = nn.Linear(enc_out, hidden_size)

    self.dropout = nn.Dropout(p=0.2)
    self.soft = nn.Softmax(dim = 2)
    self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

 # Attention mechanism
    self.beta = nn.Parameter(torch.Tensor(enc_out))
    nn.init.uniform_(self.beta, -1e-2, 1e-2)
    self.W_1 = nn.Linear(enc_out, enc_out, bias=False)
    self.W_2 = nn.Linear(hidden_size, enc_out, bias=False)

    self.W_3 = nn.Linear(hidden_size + enc_out, hidden_size, bias=False)
    self.W_out = nn.Linear(hidden_size, vocab_size, bias=False)


  def forward(self, im, label):
    enc_im = self.encode(im.to(self.device))
    dec_states, o_t = self.init_decoder(enc_im.to(self.device))
    # Suppose we are taking labels with format [Batch, Max Len, Vocab_size]
    max_len = label.shape[1]
    outputs = []
    flag = False
    for i in range(max_len):
      lll = label[:, i, :]
      target_label = torch.argmax(lll.to(self.device), dim = 1, keepdim = True)
      if flag:
        target_label = torch.argmax(outputs[-1].to(self.device), dim = 1, keepdim = True)
      # print("Target label = ", target_label.shape)

      dec_states, output = self.step_decode(dec_states, o_t.to(self.device), enc_im.to(self.device), target_label.to(self.device))
      outputs.append(output.to(self.device))
      flag = True
    outputs = torch.stack(outputs, dim=1)
    outputs = self.soft(outputs)

    return outputs




  def encode(self, im):
    cnn_encode = self.cnn_encoder(im.to(self.device))
    # print("CNN OUTPUT = ", cnn_encode.shape)
    pe = self.PositionalEmbedding2D(self.enc_out, cnn_encode.shape[2], cnn_encode.shape[3]).repeat(cnn_encode.shape[0], 1, 1, 1).to(self.device)
    # print("PE = ", pe.shape)
    # print(pe)
    cnn_encode = cnn_encode + pe
    cnn_encode = cnn_encode.permute(0, 2, 3, 1)
    batch, height, width, channel = cnn_encode.shape
    # pe_enc = self.PositionalEmbedding2D(, )


    cnn_encode = cnn_encode.contiguous().view(batch, height * width, -1)
    return cnn_encode


  def PositionalEmbedding2D(self, D_model,height,width):
    if D_model % 4 != 0:
        raise ValueError("Cannot use sin/cos positional encoding with "
                         "odd dimension (got dim={:d})".format(D_model))
    pe = torch.zeros(D_model,height,width)
    d_model = int(D_model / 2)
    div_term = torch.exp(torch.arange(0., d_model, 2) * -(math.log(10000.0) / d_model))
    pos_w = torch.arange(0., width).unsqueeze(1)
    pos_h = torch.arange(0., height).unsqueeze(1)
    pe[0:d_model:2, :, :] = torch.sin(pos_w * div_term).transpose(0, 1).unsqueeze(1).repeat(1, height, 1)
    pe[1:d_model:2, :, :] = torch.cos(pos_w * div_term).transpose(0, 1).unsqueeze(1).repeat(1, height, 1)
    pe[d_model::2, :, :] = torch.sin(pos_h * div_term).transpose(0, 1).unsqueeze(2).repeat(1, 1, width)
    pe[d_model + 1::2, :, :] = torch.cos(pos_h * div_term).transpose(0, 1).unsqueeze(2).repeat(1, 1, width)

    return pe.permute(0, 1, 2)


  def init_decoder(self, enc_im):
    enc_im_mean = enc_im.mean(dim=1)
    h = torch.tanh(self.wh_in(enc_im_mean))
    c = torch.tanh(self.wc_in(enc_im_mean))
    o_init = torch.tanh(self.wo_in(enc_im_mean))
    return (h, c), o_init



  def step_decode(self, dec_states, o_t, enc_im, target_label ):
    y_emb = self.emb(target_label).squeeze(1)
    # print("y_emb = ", y_emb.shape)
    # print("o_T = ", o_t.shape)
    inp = torch.cat([y_emb, o_t], dim = 1)
    # print("inp = ", inp.shape)
    # print("h = ", dec_states[0].shape)
    h, c = self.lstm(inp, dec_states)
    h = self.dropout(h)
    c = self.dropout(c)

    contex = self.attentionMechanism(enc_im, h)

    o_t = self.W_3(torch.cat([h, contex], dim=1)).tanh()
    o_t = self.dropout(o_t)
    output = self.W_out(o_t)

    return (h, c), output


  def attentionMechanism(self, enc_im, h):
    alpha = torch.tanh(self.W_1(enc_im) + self.W_2(h).unsqueeze(1))
    alpha = torch.sum(self.beta * alpha, dim=-1)  # [B, L]
    alpha = F.softmax(alpha, dim=-1)  # [B, L]

    # cal context: [B, C]
    contex = torch.bmm(alpha.unsqueeze(1), enc_im)
    contex = contex.squeeze(1)
    return contex



In [ ]:
a = [1,2,3,4]
a[1:2]

[2]

Initializing the database and code output

In [ ]:
'''
Initialize dataset and image preprocessing

NOTE:
    Some of the images in the dataset are corrupted. To deal with this,
    there is a check for each image to ensure that it can be loaded.
'''
import torchvision.transforms as transforms

reduced_imsize = (32, 128)  # Images are reduced to this size

# Define the transform pipeline - add normalization?
transform = transforms.Compose([
    transforms.Resize(reduced_imsize),
    transforms.ToTensor(),
])

train_dataset = LatexDataset(transform=transform, dataroot=train_root)
test_dataset = LatexDataset(transform=transform, dataroot=test_root)
val_dataset = LatexDataset(transform=transform, dataroot=val_root)

# Device settings
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}')

Device: cuda:0


In [ ]:
'''
Initialize hyperperameters, trainloader, and dictionary of LaTeX token mappings
'''

# Hyperparameters
batch_size = 64
learning_rate = 0.0003
weight_decay = 0.0003  # (L2 penalty)

max_tokens = 700        # Maximum number of tokens in a latex string
latex_dict = LatexDict(num_tokens=max_tokens)

trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=8, drop_last=True)

# print(f'Dictionary length: {latex_dict.__len__()}')
# print(f'Dictionary: {latex_dict.__dict__()}')

In [ ]:
'''
Initialize Model

Initialize Loss Functions:
1. Normal Cross-Entropy Loss between prediction and label
2. LaTeX compile test:
    - Custom function, returns True if code can compile into LaTeX, False if not

Initialize Optimizer:
1. Adam Optimizer
'''

    # Hyperparameters
# embedding_size = 80; # number of rows in the E-matrix
# o_layer_size = 700  ;  # size of o-vektorn TODO: What should this be?
# hidden_size = 700  ;
# sequence_length = 700  ; vocab_size = 700  ;
# batch_size = 16
# n_epochs = 2
# beam_size = 5

hidden_size = 100
batch_size = 64
seq_length = 300
vocab_size = 700
embedding_size = 80
model = decoder(hidden_size, batch_size, seq_length, vocab_size, embedding_size).to(device)

# model = Model(embedding_size=embedding_size, hidden_size=hidden_size, batch_size=batch_size, sequence_length=sequence_length, vocab_size=vocab_size, o_layer_size = o_layer_size).to(device)
criterion = nn.CrossEntropyLoss()   ## May need to change this
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=0)

Testing the dimensionality of CNN output for LSTM inputs

In [ ]:
pbar_test = trainloader
images, y = next(iter(pbar_test))
cnn_test_block = CNN_Block()
cnn_output = cnn_test_block(images)
print(cnn_output.shape)

torch.Size([16, 512, 2, 8])


In [ ]:
from tqdm import tqdm

'''
Training Loop
'''

num_epoch = 10
model.train()
for epoch in range(num_epoch):
    print('epoch:', epoch)
    pbar = tqdm(trainloader)
    for images, y in pbar:

        images = images.to(device)              # Send to gpu

        y_vec = latex_dict.map_tokens(list(y), batch_size=batch_size)
        #print(y_vec)
        latexx = latex_dict.tokens_to_tex(y_vec[0])
        #print(latexx)
        y_vec = torch.tensor(y_vec, dtype=torch.long)
        y_vec = torch.LongTensor(y_vec)
        one_hot = torch.nn.functional.one_hot(y_vec, num_classes=max_tokens)
        one_hot = one_hot.to(device)                # Send to gpu
        #print(y_vec[0].shape)
        #print(images.shape)
        #V = images.reshape(images.shape[0], images.shape[1] * images.shape[2], images.shape[3])
        predictions = model.forward(images,one_hot)        # Get predictions

        print(torch.argmax(predictions[1]),torch.argmax(one_hot[1]))
        print(predictions.shape,one_hot.shape)
        one_hot = torch.tensor(one_hot, dtype=torch.float)
        #print('y_vec',y_vec.shape)
        #print('pred',predictions.shape)
        loss1 = criterion(predictions, one_hot)   # Calculate first loss function
        print(loss1)
        # loss2 = can_compile(precitions)         # Check if output can compile

        optimizer.zero_grad()

        loss1.backward()
        optimizer.step()


epoch: 0


  0%|          | 0/2460 [00:00<?, ?it/s]<ipython-input-68-516763ca60d5>:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_vec = torch.tensor(y_vec, dtype=torch.long)


tensor(66454, device='cuda:0') tensor(29, device='cuda:0')
torch.Size([64, 700, 700]) torch.Size([64, 700, 700])
tensor(6.5511, device='cuda:0', grad_fn=<DivBackward1>)


<ipython-input-68-516763ca60d5>:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  one_hot = torch.tensor(one_hot, dtype=torch.float)
  0%|          | 1/2460 [00:02<1:24:04,  2.05s/it]

tensor(263154, device='cuda:0') tensor(4, device='cuda:0')
torch.Size([64, 700, 700]) torch.Size([64, 700, 700])
tensor(6.5511, device='cuda:0', grad_fn=<DivBackward1>)


...

tensor(71401, device='cuda:0') tensor(71, device='cuda:0')
torch.Size([64, 700, 700]) torch.Size([64, 700, 700])
tensor(6.5051, device='cuda:0', grad_fn=<DivBackward1>)


 52%|█████▏    | 1280/2460 [29:58<27:50,  1.42s/it]

tensor(179201, device='cuda:0') tensor(68, device='cuda:0')
torch.Size([64, 700, 700]) torch.Size([64, 700, 700])
tensor(6.4957, device='cuda:0', grad_fn=<DivBackward1>)


 52%|█████▏    | 1281/2460 [30:00<27:37,  1.41s/it]


KeyboardInterrupt: ignored